In [1]:
import os
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.llms.openai import OpenAI
from langchain.agents import AgentExecutor
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI

API_KEY = open('key.txt', 'r').read().strip()
os.environ["OPENAI_API_KEY"] = API_KEY

from dotenv import load_dotenv
load_dotenv()

False

In [3]:
SQL_PREFIX = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

If the question does not seem related to the database, just return "I don't know" as the answer.

SQL query format example:
Question: "Who are the top 5 retailers for the month of May in terms of total play time?"
Query: SELECT "Retail Name", SUM("Total Play time") as total_play_time 
       FROM "dailyLog" 
       WHERE EXTRACT(MONTH FROM "Date") = 5 
       GROUP BY "Retail Name" 
       ORDER BY total_play_time DESC 
       LIMIT 5

"""

SQL_FUNCTIONS_SUFFIX = """I should look at the tables in the database to see what I can query.  Then I should query the schema of the most relevant tables."""

FORMAT_INSTRUCTIONS = """Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question
"""

In [5]:
host="localhost"
port="5432"
database="ReportDB"
username="postgres"
password="postgres"

db = SQLDatabase.from_uri(f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}")

llm = ChatOpenAI(model_name="gpt-4", temperature=0)

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [6]:
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    prefix=SQL_PREFIX,
    suffix=SQL_FUNCTIONS_SUFFIX,
    format_instructions=FORMAT_INSTRUCTIONS,
    agent_executor_kwargs = {'return_intermediate_steps': True}          # return_full_results=True ?
)

In [4]:
def get_response(input_text):
    response = agent_executor(input_text)
    sql_query = response['intermediate_steps'][-1][0].tool_input
    message = response['intermediate_steps'][-1][0].message_log[0].content
    answer = response['output']
    return sql_query, message, answer

In [7]:
steps = agent_executor("Who are the top 5 retailers for the month of May?")



> Entering new  chain...

Invoking: `sql_db_list_tables` with ``


dailyLog
Invoking: `sql_db_schema` with `dailyLog`



CREATE TABLE "dailyLog" (
	"Date" DATE, 
	"Retail Name" TEXT, 
	"Zone" TEXT, 
	"Location" TEXT, 
	"Total Play time" INTERVAL, 
	"Efficiency (percent)" DOUBLE PRECISION, 
	"Start Time" TIMESTAMP WITHOUT TIME ZONE, 
	"End Time" TIMESTAMP WITHOUT TIME ZONE, 
	"Device Offline time" INTERVAL, 
	"Remarks" TEXT
)

/*
3 rows from dailyLog table:
Date	Retail Name	Zone	Location	Total Play time	Efficiency (percent)	Start Time	End Time	Device Offline time	Remarks
2023-03-01	GRD-004 Soudia Store	New Market Zone (75 Screens)	Green Road	12:10:35	122.0	2023-03-01 10:27:27	2023-03-01 23:19:36	0:22:00	None
2023-03-01	GRD-007 Ma genarel store	New Market Zone (75 Screens)	Green Road	11:52:08	119.0	2023-03-01 09:55:49	2023-03-01 22:05:03	0:00:00	None
2023-03-01	GRD-009 Al Karim Super Shop	New Market Zone (75 Screens)	Green Road	12:27:40	125.0	2023-03-01 09:02:04	2023-03-01 22:04:56	0:1

In [8]:
print(steps.keys())

dict_keys(['input', 'output', 'intermediate_steps'])


In [46]:
input = steps['input']
print(input)

Who are the top 5 retailers for the month of May?


In [48]:
output = steps['output']
print(output)

The top 5 retailers for the month of May are:

1. GRD-010 Ayra Mart
2. MDK-002 Hakim store
3. HBH-004 New General Balaka Store
4. ZGT-016 Brothers genarel store
5. GRD-004 Soudia Store


In [49]:
print(len(steps['intermediate_steps']))

4


In [71]:
for i in range(len(steps['intermediate_steps'])):
    print('-----------------')
    print('Step', i)
    print('-----------------')
    print('>>')
    print(steps['intermediate_steps'][i][0])
    print('>>')
    print(steps['intermediate_steps'][i][1])

-----------------
Step 0
-----------------
>>
_FunctionsAgentAction(tool='sql_db_list_tables', tool_input='', log='\nInvoking: `sql_db_list_tables` with ``\n\n\n', message_log=[AIMessage(content='', additional_kwargs={'function_call': {'name': 'sql_db_list_tables', 'arguments': '{\n  "__arg1": ""\n}'}}, example=False)])
>>
dailyLog
-----------------
Step 1
-----------------
>>
_FunctionsAgentAction(tool='sql_db_schema', tool_input='dailyLog', log='\nInvoking: `sql_db_schema` with `dailyLog`\n\n\n', message_log=[AIMessage(content='', additional_kwargs={'function_call': {'name': 'sql_db_schema', 'arguments': '{\n  "__arg1": "dailyLog"\n}'}}, example=False)])
>>

CREATE TABLE "dailyLog" (
	"Date" DATE, 
	"Retail Name" TEXT, 
	"Zone" TEXT, 
	"Location" TEXT, 
	"Total Play time" INTERVAL, 
	"Efficiency (percent)" DOUBLE PRECISION, 
	"Start Time" TIMESTAMP WITHOUT TIME ZONE, 
	"End Time" TIMESTAMP WITHOUT TIME ZONE, 
	"Device Offline time" INTERVAL, 
	"Remarks" TEXT
)

/*
3 rows from dailyLo

In [82]:
for i in range(len(steps['intermediate_steps'])):
    if steps['intermediate_steps'][i][0].message_log[0].content != '':
        print('Tool:')
        print(steps['intermediate_steps'][i][0].tool)
        print('Message:')
        print(steps['intermediate_steps'][i][0].message_log[0].content)

Tool:
sql_db_query_checker
Message:
The "dailyLog" table contains the information we need. We can group by "Retail Name" and sum the "Total Play time" for the month of May. Let's write the query.


In [77]:
if steps['intermediate_steps'][0][0].tool == 'sql_db_list_tables':
    list_of_tables = steps['intermediate_steps'][0][1]
print(list_of_tables)

dailyLog


In [41]:
if steps['intermediate_steps'][1][0].tool == 'sql_db_schema':
    schema = steps['intermediate_steps'][1][1]
print(schema)


CREATE TABLE "dailyLog" (
	"Date" DATE, 
	"Retail Name" TEXT, 
	"Zone" TEXT, 
	"Location" TEXT, 
	"Total Play time" INTERVAL, 
	"Efficiency (percent)" DOUBLE PRECISION, 
	"Start Time" TIMESTAMP WITHOUT TIME ZONE, 
	"End Time" TIMESTAMP WITHOUT TIME ZONE, 
	"Device Offline time" INTERVAL, 
	"Remarks" TEXT
)

/*
3 rows from dailyLog table:
Date	Retail Name	Zone	Location	Total Play time	Efficiency (percent)	Start Time	End Time	Device Offline time	Remarks
2023-03-01	GRD-004 Soudia Store	New Market Zone (75 Screens)	Green Road	12:10:35	122.0	2023-03-01 10:27:27	2023-03-01 23:19:36	0:22:00	None
2023-03-01	GRD-007 Ma genarel store	New Market Zone (75 Screens)	Green Road	11:52:08	119.0	2023-03-01 09:55:49	2023-03-01 22:05:03	0:00:00	None
2023-03-01	GRD-009 Al Karim Super Shop	New Market Zone (75 Screens)	Green Road	12:27:40	125.0	2023-03-01 09:02:04	2023-03-01 22:04:56	0:16:00	None
*/


In [78]:
if steps['intermediate_steps'][-1][0].tool == 'sql_db_query':
    query = steps['intermediate_steps'][-1][0].tool_input
print(query)

SELECT "Retail Name", SUM("Total Play time") as total_play_time FROM "dailyLog" WHERE EXTRACT(MONTH FROM "Date") = 5 GROUP BY "Retail Name" ORDER BY total_play_time DESC LIMIT 5


In [83]:
if steps['intermediate_steps'][-1][0].tool == 'sql_db_query':
    query_output = steps['intermediate_steps'][-1][1]
print(query_output)

[('GRD-010 Ayra Mart', None), ('MDK-002 Hakim store', datetime.timedelta(days=18, seconds=36232)), ('HBH-004 New General Balaka Store', datetime.timedelta(days=18, seconds=14340)), ('ZGT-016 Brothers genarel store', datetime.timedelta(days=17, seconds=69202)), ('GRD-004 Soudia Store', datetime.timedelta(days=17, seconds=25273))]


In [90]:
import openai

message_history = [{"role": "user", "content": f"""You are a SQL query explainer bot. That means you will explain the logic of a SQL query. 
                    There is a postgreSQL database table with the following table:

                    {schema}                   
                    
                    A SQL query is executed on the table and it returns the following result:

                    {query_output}

                    I will give you the SQL query executed to get the result and you will explain the logic executed in the query.
                    Make the explanation brief and simple. It will be used as the explanation of the results. Do not mention the query itself.
                    No need to explain the total query. Just explain the logic of the query.
                    Reply only with the explaination to further input. If you understand, say OK."""},
                   {"role": "assistant", "content": f"OK"}]

In [93]:
def explain(query):
    global message_history

    message_history.append({"role": "user", "content": query})

    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=message_history,
    )

    reply_content = completion.choices[0].message.content
    print("Explaination:\n"+ reply_content)

In [94]:
explain(query)

Explaination:
The query selects the "Retail Name" column and calculates the total play time by summing up the "Total Play time" column. It filters the data by only considering records where the month of the "Date" column is 5. Then, it groups the result by "Retail Name" column. Finally, it sorts the result in descending order based on the total play time and limits the output to the top 5 records.
